In [ ]:
import pandas as pd
import datetime, pytz
import random

### Read users

In [ ]:
users = pd.read_excel("../../data/users_processed.xlsx")

In [ ]:
users

In [ ]:
## Shuffle users 
users = users.sample(frac=1).reset_index(drop=True)

### Read workflow def

In [ ]:
teams_df = pd.read_excel("../../data/workflow.xlsx", sheet_name="teams")
teams = teams_df['Team'].tolist()

steps_df = pd.read_excel("../../data/workflow.xlsx", sheet_name="steps")
step_names = steps_df['Step'].tolist()

users_df = pd.read_excel("../../data/users_processed.xlsx")


In [ ]:
tz_list = users_df['tz'].unique()


In [ ]:
state_of_world = {
    'current_time': datetime.datetime(2005, 1, 1, 1, 0, 0, 0, tzinfo=datetime.timezone.utc),
    'tzs': tz_list,
    'tasks' : None,
    'users' : users,
    'user_tasks' : {},
    'task_id_cntr' : 0,
    'task_instance_id_cntr' : 0,
}

In [ ]:
def get_rows_by_val(df, col, val, reset_index=True):
    temp_df = df.loc[df[col] == val]
    if reset_index:
        temp_df.reset_index(drop=True, inplace=True)
    return temp_df    

In [ ]:
def get_users_by_team(user_df, team):
    temp_df = get_rows_by_val(user_df, 'team', team)
    return temp_df

In [ ]:
def read_task_def(state,step_names):
    team_col_idx = 1
    steps={}
    for i in range(len(step_names)):
        step = {}
        step_name = step_names[i]
        if i == len(step_names)-1:
            next_step = None
        else:
            next_step = step_names[i+1]
        step["next_step"]=next_step

        row_df = steps_df.loc[steps_df['Step'] == step_name]
        team_name = row_df.iat[0,1]
        step["team"] = team_name
        steps[step_name]=step    
    state['tasks']  = steps
    

In [ ]:
read_task_def(state_of_world,step_names)



In [ ]:
state_of_world

### Generate tasks for step 1 

In [ ]:
def get_users_for_task(task_instance, state):
    task_type = task_instance['task_type']
    task = state['tasks'][task_type]
    team = task['team']
    elig_users = get_users_by_team(state['users'], team)
    return elig_users

In [ ]:
def get_next_task_step(state,current_step):
    if current_step is None:
        step = step_names[0]
    else:
        step = state['tasks'][current_step]["next_step"]
    return step

In [ ]:



def generate_and_assign_workflow_action(state, prev_inst=None):
    inst={}
    if prev_inst is None:
        next_step = get_next_task_step(state,None)
        state['task_id_cntr'] += 1
        task_id = state['task_id_cntr']
        prev_task_instance_id = None
    else:
        next_step = get_next_task_step(state,prev_inst['task_type'])
        task_id = prev_inst['task_id']
        prev_task_instance_id = prev_inst['task_instance_id']
    if next_step is None:
        #Do nothing
        return None
    else:
        inst['prev_task_instance_id'] = prev_task_instance_id
        task_instance_id = state['task_instance_id_cntr']
        state['task_instance_id_cntr'] += 1
        inst['tz'] = random.choice(state['tzs'])
        #Find all users who can execute this task
        inst['task_id'] = task_id
        inst['task_instance_id'] = task_instance_id
        if prev_inst is not None:
            prev_inst['next_task_instance_id'] = task_instance_id
        inst['task_type'] = next_step
        inst['task_type'] = next_step
        elig_users = get_users_for_task(inst , state)
        user_ids = elig_users['id'].unique()
        inst['user_id'] = random.choice(user_ids)
        inst['creation_time_utc'] = state['current_time']
        inst['next_task_instance_id'] = None
        inst['start_time_utc'] = None
        inst['stop_time_utc'] = None
        return inst



In [ ]:
state = state_of_world

In [ ]:
task = None
task_instances = []
for i in range(100):
    task = generate_and_assign_workflow_action(state,task)
    if task is not None:
        task_instances.append(task)

In [ ]:

t_inst_df = pd.DataFrame(task_instances)

In [ ]:
t_inst_df
